<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main load

In [0]:
import os,subprocess, re

# system functions
def recursive_glob(treeroot, pattern):
  results = []
  for base, dirs, files in os.walk(treeroot):
    goodfiles = fnmatch.filter(files, pattern)
    results.extend(os.path.join(base, f) for f in goodfiles)
  return results

def cmdline(cmd,pr=False):
  result =[]
  f = os.popen(cmd)
  try:
    for line in f:
      if pr == True:
        print(line)
        result.append(line),
  finally:
    f.close()
  return result

# stinking default folder...poef!
cmdline('rm -r /content/sample_data')
# cmdline('pip install gitpython')
# pick fs
filesystem = "local"
if filesystem == 'gdrive':
  # sync google drive
  from google.colab import drive
  import os
  # if drive is needed uncomment
  print('Google drive ',end='')
  drive.mount('/content/drive',force_remount=True)
  print('Root folder set to ',end='')
  root = '/content/drive/My drive/image_learning'
  print(root)
else:
  print('Root folder set to ',end='')
  root = '/content/image_learning'
  print(root) 

#shoot root
os.makedirs(root,exist_ok = True)
os.chdir(root)

# install missing libs
cmdline('pip install sklearn imageio tensorflow-gpu==2.0.0-alpha0')
cmdline('apt install tree')
cmdline('pip install -q --upgrade --force-reinstall tensorflow-gpu')

# import the rest
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.optimizers import SGD, RMSprop,adam
from keras.utils import np_utils
import theano, glob, shutil, fnmatch, time, cv2, itertools
from IPython.display import clear_output, display, Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from subprocess import PIPE, Popen
import tensorflow as tf

# list of reps uncomment to install
reps=[
#         'drallensmith/neat-python',
#         'CodeReclaimers/neat-python',
#         'uber-research/deep-neuroevolution',
#         'MorvanZhou/Evolutionary-Algorithm',
#         'llSourcell/blockchain-python-tutorial',
#         'llSourcell/autoencoder_demo',
#         'llSourcell/autoencoder_explained',
#         'llSourcell/awesome-public-datasets',
#         'llSourcell/7_Research_Directions_Deep_Learning',
#         'hardmaru/sketch-rnn',
#         'hardmaru/sketch-rnn-datasets',
#         'hardmaru/sketch-rnn-poste',
#         'Gogul09/virtual-drum',
#         'ytdl-org/youtube-dl',
#         'nnUyi/GAN-Collections',
#         'carpedm20/BEGAN-tensorflow',
        'mikf/gallery-dl',
#         'NVlabs/stylegan',
#         'mauriceling/dose',
#         'corenel/Realistic-Neural-Talking-Head-Models',
        'bxck75/piss-ant-pix2pix',
#         'bxck75/datasets',
#         'hardmaru/estool',
#         'deepmipt/DeepPavlov',
#         'bxck75/A1_Colabs',
#         'tjwei/Flappy-Turtle.',
#         'tjwei/fonttools',
#         'tjwei/blender3d_import_psk_psa',
#         'lllyasviel/sketchKeras',
#         'Mckinsey666/Anime-Face-Dataset',
#         'chenyuntc/pytorch-book',
#         'lllyasviel/style2paints',
#         'llSourcell/GANS-for-style-transfer',
#         'opencv/open_model_zoo',
#         'hindupuravinash/the-gan-zoo',
#         'corenel/GAN-Zoo',
#         'eriklindernoren/Keras-GAN',
#         'junyanz/CycleGAN',
#         'junyanz/pytorch-CycleGAN-and-pix2pix',
#         'junyanz/iGAN', #----> !wget http://efrosgans.eecs.berkeley.edu/iGAN/datasets/church_64.zip <----dataset 	outdoor_128.zip 	handbag_128.zip !!!
#         'martinarjovsky/WassersteinGAN',
#         'shaoanlu/faceswap-GAN',
#         'LantaoYu/SeqGAN',
#         'tjwei/GANotebooks',
#         'adeshpande3/Tensorflow-Programs-and-Tutorials',
#         'adeshpande3/Generative-Adversarial-Networks',
#         'adeshpande3/KaggleGhosts',
#         'adeshpande3/OpenAI_Gym_Universe',
#         'diegoalejogm/gans',
#         'osh/KerasGAN',
#         'r9y9/gantts',
#         'jayleicn/animeGAN',
#         'jayleicn/ImageNet-Training',
#         'Zardinality/WGAN-tensorflow',
#         'tree-gan/BonsaiNet',
#         'RossMcKenzie/TreeGAN',
#         'carpedm20/BEGAN-tensorflow',
#         'martinarjovsky/WassersteinGAN',
#         'timsainb/Tensorflow-MultiGPU-VAE-GAN',
#         'Larox/python-moviepy-meetup',
#         'tjwei/keras-yolo3',
#         'tensorflow/gan',
#         'tensorflow/moonlight'
#         'tensorflow/models',
#         'tensorflow/datasets',
#         'tensorflow/docs',
#         'mnicnc404/CartoonGan-tensorflow',
#         'Yijunmaverick/CartoonGAN-Test-Pytorch-Torch',
#         'keras-team/keras-contrib',
#         'mnicnc404/CartoonGan-tensorflow',
]

def findFilesInFolderYield(path,  extension, containsTxt='', subFolders = True, excludeText = ''):

    if type(containsTxt) == str: # if a string and not in a list
        containsTxt = [containsTxt]

    myregexobj = re.compile('\.' + extension + '$')    # Makes sure the file extension is at the end and is preceded by a .
    try:   # Trapping a OSError or FileNotFoundError:  File permissions problem I believe
        for entry in os.scandir(path):
            if entry.is_file() and myregexobj.search(entry.path): # 
                bools = [True for txt in containsTxt if txt in entry.path and (excludeText == '' or excludeText not in entry.path)]
                if len(bools)== len(containsTxt):
                    yield entry.stat().st_size, entry.stat().st_atime_ns, entry.stat().st_mtime_ns, entry.stat().st_ctime_ns, entry.path
            elif entry.is_dir() and subFolders:   # if its a directory, then repeat process as a nested function
                yield from findFilesInFolderYield(entry.path,  extension, containsTxt, subFolders)
    except OSError as ose:
        print('Cannot access ' + path +'. Probably a permissions error ', ose)
    except FileNotFoundError as fnf:
        print(path +' not found ', fnf)        
# Gitgo class
class GitGo(): 
  def __init__(self,repos,sub_repos=False,chdir=True,path='/content/'):
    self.sub_repo_list = []
    self.GitUsers=[]
    self.repos = repos
    self.chdir = chdir
    self.path = path
    os.makedirs(self.path, exist_ok = True)
    if 'help' in self.repos:
      self.help()
    self.pull_reps()
    self.custom_reps_setup()
    if sub_repos == True:
      self.get_other_reps()
  def help(self):
    return "* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo(<list of reps to install>, sub_repos=<True/False, chdir=<True/False>, path=<root path>)\
    #--pull al selected reps  
  def pull_reps(self):    
    for rep in self.repos:
      self.rep=rep.split('/')
      # change folder check
      if self.chdir ==True:
        #Switch to path
        os.chdir(self.path)
      # pull the git repo
      cmdline('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git',True)
      # Set the return value for rep rootpath
      self.PATH=self.path+self.rep[1]
    # show imported files
    os.system('ls ' + root)  
    #--repository required dependancies setup
  def custom_reps_setup(self):
    # cyclegan
    if 'pytorch-CycleGAN-and-pix2pix' in self.repos:
      !export PYTHONPATH={root}/pytorch-CycleGAN-and-pix2pix:$PYTHONPATH
      !pip install dominate
    # custom stuff for CartoonGAN-tensorflow and keras-team/keras-contrib   
    if 'keras-team/keras-contrib' in self.repos:
      os.chdir(self.path+'/keras-contrib')
      cmdline('python convert_to_tf_keras.py')
      cmdline('USE_TF_KERAS=1')
      cmdline('python setup.py install')
      import tensorflow as tf
      tf.__version__     
    # custom setup stuff for gallery-dl repo   
    if 'mikf/gallery-dl' in self.repos:
      os.chdir(root+'/gallery-dl')
      cmdline("pip install -e . |grep 'succes'",True)
    # custom setup stuff for youtube-dl repo    
    if 'ytdl-org/youtube-dl' in self.repos:
      os.chdir(root+'/youtube-dl') 
      cmdline("pip install -e . |grep 'succes'",True)      
    # switch backt to root
    os.chdir(self.path)    
#--grab the username if a repos is installed
#--generate a file of all other reps of users
  def get_other_reps(self):          
    for r in self.repos:
      self.GUSER=r.split('/')[0]
      self.repo_name=r.split('/')[1]
      self.GitUsers.append(self.GUSER)      
      !curl https://api.github.com/users/{self.GUSER}/repos?per_page=100 \
            | grep -o 'git@[^"]*' > {root}/info.txt
      cmdline('cat '+root+"/info.txt |awk -F ':' '{print $2}'|awk -F '.' '{print $1}' \
          > "+self.path+"/"+self.GUSER+"_repositories.txt",True)      
      with open(root+'/info.txt','r') as f:
        for line in f:
          cline=line.split(':')[1].split('.')[0]
          self.sub_repo_list.append(cline),  
            #--return path and sub reps
  def __repr__(self):
    return self.path,self.sub_repo_list
# Hauling Ass!
G,R=GitGo(reps,sub_repos=True,path=root)
loot = recursive_glob(root,'req*.txt')

clear_output()

In [0]:
keyword='turtle'
logdir=root+'/LOG'
datadir=root+'/DATA'
checkpointdir='/CHECKPOINT'
!mkdir     {checkpointdir}
!mkdir     {datadir}
!mkdir     {logdir}
# Scrape the crap!
!gallery-dl -d /content/image_learning/DATA \
    --range 1-25 https://flickr.com/search/?text={keyword}
# Initialization tensorboard
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Add to tf.keras callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
!bash {root}/piss-ant-pix2pix/image_folder_make_set_train.sh \
      {root}/DATA/flickr/search/turtle 10 prep





In [0]:
# /content/image_learning/DATA/flickr/search/turtle
!cat /content/image_learning/piss-ant-pix2pix/image_folder_make_set_train.sh

In [0]:
os.chdir(root+'/'+piss-ant-pix2pix)

if train:
  cmdline('python pix2pix.py \
      --mode train \
      --output_dir '+input_img_folder+'_train \
      --max_epochs '+training_epochs+' \
      --input_dir '+input_img_folder+'_resize/_combined/train \
      --which_direction BtoA &')

if test:
  cmdline('python pix2pix.py \
      --mode test \
      --output_dir '+input_img_folder+'_test \
      --input_dir '+input_img_folder+'_resize/_combined/val \
      --checkpoint '+input_img_folder+'_train &')






# Experimenteel

In [0]:

# GitGo(['tensorflow/tensorboard-plugin-example'],sub_repos=True,path=root)
! python /content/image_learning/tensorboard-plugin-example/greeter_tensorboard/main.py

In [0]:
%cd /content/image_learning/Keras-GAN
!pip install -r requirements.txt
import os

In [0]:
!python /content/image_learning/examples/tensorflow_examples/models/pix2pix/data_download.py

In [0]:
# os.chdir('discogan/images')

!wget 'https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/cityscapes.tar.gz' -O /content/image_learning/Keras-GAN/discogan/images/cityscapes.tar.gz

!tar -zxvf cityscapes.tar.gz

!ls -la
os.chdir(`root)

In [0]:
G_N='discogan'
# os.chdir(G_N+'/images/')

!wget 'https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/cityscapes.tar.gz' -s -O /content/image_learning/Keras-GAN/discogan/images/cityscapes.tar.gz
!tar -zxf cityscapes.tar.gz
os.chdir('../')
loot = recursive_glob(root,'downl*.sh')
for l in loot:
  x=l.split('.')[0]
  print(x)
  os.chdir(x)
  !ls -l
  print(x.rstrip())

#   !bash {l} facades maps ae_photos

# os.chdir(root+'/Keras-GAN')
# print(loot)

apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos"


In [0]:
%cd /content/image_learning/datasets/tensorflow_datasets/video
# Start TensorBoard within the notebook using magics function
%tensorboard — logdir logs

In [2]:
%tensorboard — logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf

# tfds works in both Eager and Graph modes
tf.enable_eager_execution()

# See available datasets
print(tfds.list_builders())
print(dir(tfds))
# Construct a tf.data.Dataset
ds_train, ds_test = tfds.load(name="oxford_flowers102", split=["train", "test"])

# Build your input pipeline
ds_train = ds_train.shuffle(1000).batch(128).prefetch(10)
for features in ds_train.take(1):
  image, label = features["image"], features["label"]

In [0]:
# x= '/content/image_learning/CycleGAN/pretrained_models/download_model.sh'
# print(l.split('.')[0]) 
!cat /content/image_learning/examples/tensorflow_examples/models/nmt_with_attention/distributed_train.py


In [0]:
!cat /content/image_learning/examples/tensorflow_examples/models/nmt_with_attention/nmt.py

# DCGAN

In [0]:
%%writefile /content/image_learning/examples/tensorflow_examples/models/dcgan/dcgan_custom.py
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Train.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
import tensorflow as tf # TF2
from tensorflow_examples.models.nmt_with_attention import nmt
from tensorflow_examples.models.nmt_with_attention import utils
assert tf.__version__.startswith('2')


class Train(object):
  """Train class.

  Attributes:
    epochs: Number of epochs.
    enable_function: Decorate function with tf.function.
    encoder: Encoder.
    decoder: Decoder.
    inp_lang: Input language tokenizer.
    targ_lang: Target language tokenizer.
    batch_size: Batch size.
    per_replica_batch_size: Batch size per replica for sync replicas. Same as
      batch_size for non distributed training.
    optimizer: Optimizer.
    loss_object: Object of the loss class.
    train_loss_metric: Mean metric to keep track of the train loss value.
    test_loss_metric: Mean metric to keep track of the test loss value.
  """

  def __init__(self, epochs, enable_function, encoder, decoder, inp_lang,
               targ_lang, batch_size, per_replica_batch_size):
    self.epochs = epochs
    self.enable_function = enable_function
    self.encoder = encoder
    self.decoder = decoder
    self.inp_lang = inp_lang
    self.targ_lang = targ_lang
    self.batch_size = batch_size
    self.per_replica_batch_size = per_replica_batch_size
    self.optimizer = tf.keras.optimizers.Adam()
    self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    self.train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
    self.test_loss_metric = tf.keras.metrics.Mean(name='test_loss')

  def loss_function(self, real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = self.loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) * 1. / self.batch_size

  def train_step(self, inputs):
    """One train step.

    Args:
      inputs: tuple of input tensor, target tensor.
    """

    loss = 0
    enc_hidden = self.encoder.initialize_hidden_state()

    inp, targ = inputs

    with tf.GradientTape() as tape:
      enc_output, enc_hidden = self.encoder(inp, enc_hidden)
      dec_hidden = enc_hidden
      dec_input = tf.expand_dims(
          [self.targ_lang.word_index['<start>']] * self.per_replica_batch_size,
          1)

      for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden, _ = self.decoder(
            dec_input, dec_hidden, enc_output)
        loss += self.loss_function(targ[:, t], predictions)
        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    variables = (self.encoder.trainable_variables +
                 self.decoder.trainable_variables)
    gradients = tape.gradient(loss, variables)
    self.optimizer.apply_gradients(zip(gradients, variables))

    self.train_loss_metric(batch_loss)

  def test_step(self, inputs_test):
    """One test step.

    Args:
      inputs_test: tuple of input tensor, target tensor.
    """

    loss = 0
    enc_hidden = self.encoder.initialize_hidden_state()

    inp_test, targ_test = inputs_test

    enc_output, enc_hidden = self.encoder(inp_test, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims(
        [self.targ_lang.word_index['<start>']] * self.per_replica_batch_size,
        1)

    for t in range(1, targ_test.shape[1]):
      predictions, dec_hidden, _ = self.decoder(
          dec_input, dec_hidden, enc_output)
      loss += self.loss_function(targ_test[:, t], predictions)

      prediction_id = tf.argmax(predictions, axis=1)
      # passing the predictions back to the model as the input.
      dec_input = tf.expand_dims(prediction_id, 1)

    batch_loss = (loss / int(targ_test.shape[1]))

    self.test_loss_metric(batch_loss)

  def training_loop(self, train_ds, test_ds):
    """Custom training and testing loop.

    Args:
      train_ds: Training dataset
      test_ds: Testing dataset

    Returns:
      train_loss, test_loss
    """

    if self.enable_function:
      self.train_step = tf.function(self.train_step)
      self.test_step = tf.function(self.test_step)

    template = 'Epoch: {}, Train Loss: {}, Test Loss: {}'

    for epoch in range(self.epochs):
      self.train_loss_metric.reset_states()
      self.test_loss_metric.reset_states()

      for inp, targ in train_ds:
        self.train_step((inp, targ))

      for inp_test, targ_test in test_ds:
        self.test_step((inp_test, targ_test))

      print (template.format(epoch,
                             self.train_loss_metric.result().numpy(),
                             self.test_loss_metric.result().numpy()))

    return (self.train_loss_metric.result().numpy(),
            self.test_loss_metric.result().numpy())


def run_main(argv):
  del argv
  kwargs = utils.flags_dict()
  main(**kwargs)


def main(epochs, enable_function, buffer_size, batch_size, download_path,
         num_examples=70000, embedding_dim=256, enc_units=1024, dec_units=1024):
  file_path = utils.download(download_path)
  train_ds, test_ds, inp_lang, targ_lang = utils.create_dataset(
      file_path, num_examples, buffer_size, batch_size)
  vocab_inp_size = len(inp_lang.word_index) + 1
  vocab_tar_size = len(targ_lang.word_index) + 1

  encoder = nmt.Encoder(vocab_inp_size, embedding_dim, enc_units, batch_size)
  decoder = nmt.Decoder(vocab_tar_size, embedding_dim, dec_units)

  train_obj = Train(epochs, enable_function, encoder, decoder,
                    inp_lang, targ_lang, batch_size, batch_size)
  print ('Training ...')
  return train_obj.training_loop(train_ds, test_ds)

if __name__ == '__main__':
  utils.nmt_flags()
  app.run(run_main)